<a href="https://colab.research.google.com/github/JoaoAssalim/Algorithms/blob/main/sentiment_analysis_tensorflow_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle Project: ❤️ vs 😡: Sentiment Analysis 📝

Link to page: https://www.kaggle.com/datasets/mohidabdulrehman/vs-sentiment-analysis/data

In [ ]:
!python -m spacy download en_core_web_sm

In [153]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Input, Dropout, Bidirectional
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords

import pandas as pd
import spacy

In [154]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [155]:
df = pd.read_csv("/content/Beginner_Reviews_dataset.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,sentence,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [156]:
### In this step, I made an function that tokenize all the data and
### lemmatize to get the radical from all the words and remove stopwords
### to make the sentences more clean

nlp = spacy.load("en_core_web_sm")

def custom_lemmatizer(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_.lower() for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 1]
    return " ".join(lemmatized_words)

In [157]:
df["sentence"] = df["sentence"].apply(custom_lemmatizer)

In [158]:
df = df.sample(frac=1)

In [159]:
df.head()

,sentence,label
744,back,0
658,fun experience,1
454,last time lunch bad,0
875,hardly meat,0
458,good tater tot southwest,1


In [160]:
X = df['sentence']
y = df['label']

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

> Output Modes:

* output_mode='int':
    ```
    Mapeia cada palavra para um índice inteiro.
    O texto "Eu gosto de programar em Python" pode
    ser transformado em algo como [2, 3, 4, 5, 6].
    ```

* output_mode='binary':
    ```
    Produz um vetor binário indicando a presença
    ou ausência de palavras do vocabulário no texto.
    Por exemplo, [1, 0, 0, 1, 1, 0].
    ```

* output_mode='count':
    ```
    Conta a frequência de cada palavra no vocabulário.
    Um texto pode ser transformado em um vetor como
    [1, 0, 0, 2, 1, 0] indicando que algumas palavras
    aparecem duas vezes.
    ```

* output_mode='tf-idf':
    ```
    Produz vetores TF-IDF, que consideram tanto a frequência
    de uma palavra em um documento quanto a frequência
    dessa palavra em todo o corpus. O output pode ser
    um vetor como [0.5, 0, 0, 0.5, 0.3, 0].
    ```

In [162]:
# É um vetorizador no qual tem as funcionalidades mencionadas a cima para
# vetorizar textos e assim se tornarem compativeis para o treinamento nos
# modelos que temos para NLP
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int'
)

vectorizer.adapt(X_train)

In [163]:
X_train = vectorizer(X_train)
X_test = vectorizer(X_test)

In [164]:
word_index = vectorizer.get_vocabulary()

In [177]:
# Sequential é um meio de construir modelos com tensorflow
model = Sequential([
    # Embedding é uma camada necessaria para entrada de dados quando se trata de NLP
    # transforma índices inteiros em vetores de tamanho fixo, onde cada palavra ou token é representado por um vetor denso e contínuo.

    Embedding(input_dim=len(word_index) + 1, output_dim=16),

    # capturar melhor o contexto de uma sequência de entrada, aproveitando informações
    # tanto do passado quanto do futuro em cada ponto de tempo

    Bidirectional(LSTM(128, return_sequences=True)),

    # é uma técnica de regularização usada em redes neurais para reduzir o
    # overfitting durante o treinamento. Overfitting ocorre quando um modelo se
    # ajusta muito bem aos dados de treinamento, mas não generaliza bem para novos dados não vistos.

    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=True)),

    # é usada principalmente como camada de saída ou em camadas intermediárias de uma rede neural,
    # onde cada neurônio calcula uma combinação linear das entradas da camada anterior, seguida de
    # uma função de ativação opcional.

    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [178]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [179]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64)

Epoch 1/20
13/13 [==============================] - 11s 133ms/step - loss: 0.6934 - accuracy: 0.4974
Epoch 2/20
13/13 [==============================] - 2s 174ms/step - loss: 0.6913 - accuracy: 0.5528
Epoch 3/20
13/13 [==============================] - 3s 252ms/step - loss: 0.5661 - accuracy: 0.7196
Epoch 4/20
13/13 [==============================] - 2s 128ms/step - loss: 0.2794 - accuracy: 0.8989
Epoch 5/20
13/13 [==============================] - 2s 126ms/step - loss: 0.1449 - accuracy: 0.9555
Epoch 6/20
13/13 [==============================] - 2s 129ms/step - loss: 0.0862 - accuracy: 0.9731
Epoch 7/20
13/13 [==============================] - 2s 127ms/step - loss: 0.0615 - accuracy: 0.9786
Epoch 8/20
13/13 [==============================] - 2s 127ms/step - loss: 0.0522 - accuracy: 0.9817
Epoch 9/20
13/13 [==============================] - 2s 134ms/step - loss: 0.0494 - accuracy: 0.9852
Epoch 10/20
13/13 [==============================] - 3s 269ms/step - loss: 0.0369 - accuracy: 0.992

In [180]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia: {accuracy * 100:.2f}%')

7/7 [==============================] - 2s 27ms/step - loss: 1.2998 - accuracy: 0.7179
Acurácia: 71.79%
